In [ ]:
print('Importing packages...')
import os
import urllib
import pandas
import zipfile
from IPython.display import display, HTML
os.chdir('/home/idies/workspace/Storage/raddick/raddick_cra')
#os.getcwd()

# Get oroginal datafiles from ffiec.gov
print('Downloading master datafile...')
thatpath = 'https://www.ffiec.gov/cra/xls/'
thefilename = '16exp_discl.zip'
with urllib.request.urlopen(thatpath+thefilename) as response:
    it = response.read()
    with open(thefilename, 'wb') as f:
        f.write(it)
        
print('Unzipping individual datafiles...')
thezipfile = zipfile.ZipFile(thefilename)
thezipfile.extractall()
thezipfile.close()
print('Done')


In [ ]:
# Original file is one huge giant string. Read that string.
print('Reading table D1-1...')
bizorig = pandas.read_csv('cra2016_Discl_D11.dat', header=None)#, dtype={'thestring': object})
bizorig.columns = ['thestring']

# Now parse the strings into individual values, following the data guide in the PDF
print('Parsing strings...')
bizorig['tableID'] = bizorig['thestring'].apply(lambda x: x[0:5])
bizorig['respondentID'] = bizorig['thestring'].apply(lambda x: x[5:15])
bizorig['agency_code'] = bizorig['thestring'].apply(lambda x: x[15])
bizorig['activity_year'] = bizorig['thestring'].apply(lambda x: x[16:20])
bizorig['loan_type'] = bizorig['thestring'].apply(lambda x: x[20])
bizorig['action_taken_type'] = bizorig['thestring'].apply(lambda x: x[21])
bizorig['state'] = bizorig['thestring'].apply(lambda x: x[22:24])
bizorig['county'] = bizorig['thestring'].apply(lambda x: x[24:27])
bizorig['msa'] = bizorig['thestring'].apply(lambda x: x[27:32])
bizorig['assessment_area_number'] = bizorig['thestring'].apply(lambda x: x[33:36])
bizorig['partial_county_indicator'] = bizorig['thestring'].apply(lambda x: x[36])
bizorig['split_county_indicator'] = bizorig['thestring'].apply(lambda x: x[37])
bizorig['population_classification'] = bizorig['thestring'].apply(lambda x: x[38])
bizorig['income_group_total'] = bizorig['thestring'].apply(lambda x: x[39:42])
bizorig['report_level'] = bizorig['thestring'].apply(lambda x: x[42:45])

bizorig['nBizLoans1'] = bizorig['thestring'].apply(lambda x: x[45:55])
bizorig['amtBizLoans1'] = bizorig['thestring'].apply(lambda x: x[55:65])
bizorig['nBizLoans100k'] = bizorig['thestring'].apply(lambda x: x[65:75])
bizorig['amtBizLoans100k'] = bizorig['thestring'].apply(lambda x: x[75:85])
bizorig['nBizLoans250k'] = bizorig['thestring'].apply(lambda x: x[85:95])
bizorig['amtBizLoans250k'] = bizorig['thestring'].apply(lambda x: x[95:105])
bizorig['nBizLoans1M'] = bizorig['thestring'].apply(lambda x: x[105:115])
bizorig['amtBizLoans1M'] = bizorig['thestring'].apply(lambda x: x[115:125])
bizorig['nBizLoansAff'] = bizorig['thestring'].apply(lambda x: x[125:135])
bizorig['amtBizLoansAff'] = bizorig['thestring'].apply(lambda x: x[135:145])

bizorig = bizorig.drop('thestring', axis=1)

bizorig_bk = bizorig

bizorig.sample(2)

In [ ]:
bizorig = bizorig_bk

print('Converting geography codes...')
bizorig['respondentID'] = pandas.to_numeric(bizorig['respondentID'], downcast='integer', errors='coerce')
bizorig['agency_code'] = pandas.to_numeric(bizorig['agency_code'], downcast='integer', errors='coerce')
bizorig['activity_year'] = pandas.to_numeric(bizorig['activity_year'], downcast='integer', errors='coerce')
bizorig['loan_type'] = pandas.to_numeric(bizorig['loan_type'], downcast='integer', errors='coerce')
bizorig['action_taken_type'] = pandas.to_numeric(bizorig['action_taken_type'], downcast='integer', errors='coerce')
bizorig['state'] = pandas.to_numeric(bizorig['state'], downcast='integer', errors='coerce')
bizorig['county'] = pandas.to_numeric(bizorig['county'], downcast='integer', errors='coerce')
bizorig['msa'] = pandas.to_numeric(bizorig['msa'], downcast='integer', errors='coerce')
#bizorig['assessment_area_number'] = pandas.to_numeric(bizorig['assessment_area_number'], downcast='integer', errors='coerce')

print('Converting numbers...')
bizorig['nBizLoans1'] = pandas.to_numeric(bizorig['nBizLoans1'])
bizorig['amtBizLoans1'] = pandas.to_numeric(bizorig['amtBizLoans1']) * 1000
bizorig['nBizLoans100k'] = pandas.to_numeric(bizorig['nBizLoans100k'])
bizorig['amtBizLoans100k'] = pandas.to_numeric(bizorig['amtBizLoans100k']) * 1000
bizorig['nBizLoans250k'] = pandas.to_numeric(bizorig['nBizLoans250k'])
bizorig['amtBizLoans250k'] = pandas.to_numeric(bizorig['amtBizLoans250k']) * 1000
bizorig['nBizLoans1M'] = pandas.to_numeric(bizorig['nBizLoans1M'])
bizorig['amtBizLoans1M'] = pandas.to_numeric(bizorig['amtBizLoans1M']) * 1000
bizorig['nBizLoansAff'] = pandas.to_numeric(bizorig['nBizLoansAff'])
bizorig['amtBizLoansAff'] = pandas.to_numeric(bizorig['amtBizLoansAff']) * 1000

bizorig_bk = bizorig
print('ok')


In [ ]:
bizorig = bizorig_bk

print('Looking up data codes...')
# agency code
bizorig['agency'] = ''
bizorig.loc[bizorig['agency_code'] == 1, 'agency'] = 'OCC'
bizorig.loc[bizorig['agency_code'] == 2, 'agency'] = 'FRS'
bizorig.loc[bizorig['agency_code'] == 3, 'agency'] = 'FDIC'
bizorig.loc[bizorig['agency_code'] == 4, 'agency'] = 'OTS'

# income group code
bizorig['income_group'] = 'xxx'
bizorig.loc[bizorig['income_group_total'] == '001', 'income_group'] = '< 10% MFI'
bizorig.loc[bizorig['income_group_total'] == '002', 'income_group'] = '10-20% MFI'
bizorig.loc[bizorig['income_group_total'] == '003', 'income_group'] = '20-30% MFI'
bizorig.loc[bizorig['income_group_total'] == '004', 'income_group'] = '30-40% MFI'
bizorig.loc[bizorig['income_group_total'] == '005', 'income_group'] = '40-50% MFI'
bizorig.loc[bizorig['income_group_total'] == '006', 'income_group'] = '50-60% MFI'
bizorig.loc[bizorig['income_group_total'] == '007', 'income_group'] = '60-70% MFI'
bizorig.loc[bizorig['income_group_total'] == '008', 'income_group'] = '70-80% MFI'
bizorig.loc[bizorig['income_group_total'] == '009', 'income_group'] = '80-90% MFI'
bizorig.loc[bizorig['income_group_total'] == '010', 'income_group'] = '90-100% MFI'
bizorig.loc[bizorig['income_group_total'] == '011', 'income_group'] = '100-110% MFI'
bizorig.loc[bizorig['income_group_total'] == '012', 'income_group'] = '110-120% MFI'
bizorig.loc[bizorig['income_group_total'] == '013', 'income_group'] = '> 120% MFI'
bizorig.loc[bizorig['income_group_total'] == '013', 'income_group'] = '> 120% MFI'
bizorig.loc[bizorig['income_group_total'] == '014', 'income_group'] = 'MFI not known'
bizorig.loc[bizorig['income_group_total'] == '015', 'income_group'] = 'Tract not known'
bizorig.loc[bizorig['income_group_total'] == '101', 'income_group'] = 'Low income'
bizorig.loc[bizorig['income_group_total'] == '102', 'income_group'] = 'Moderate income'
bizorig.loc[bizorig['income_group_total'] == '103', 'income_group'] = 'Middle income'
bizorig.loc[bizorig['income_group_total'] == '104', 'income_group'] = 'Upper income'
bizorig.loc[bizorig['income_group_total'] == '105', 'income_group'] = 'Income not known'
bizorig.loc[bizorig['income_group_total'] == '106', 'income_group'] = 'Tract not known (via cra level)'

# Find institution names by respondent IDs
respondents = pandas.read_csv('respondentid.csv', index_col='respondentID')
bizorig = bizorig.join(respondents, how='left', on='respondentID')

print('Summing numbers...')
bizorig['nBizLoans'] = bizorig['nBizLoans1'] + bizorig['nBizLoans100k'] + bizorig['nBizLoans250k'] #+ bizorig['nBizLoans1M']# + bizorig['nBizLoansAff']
bizorig['amtBizLoans'] = bizorig['amtBizLoans1'] + bizorig['amtBizLoans100k'] + bizorig['amtBizLoans250k'] #+ bizorig['amtBizLoans1M']# + bizorig['amtBizLoansAff']


bizorig.index.name = 'row_number'

# Remove county totals (blank income group) so we don't double-count
bizorig = bizorig[(bizorig['income_group'] != 'xxx')]

#bizorig.groupby('income_group').size()

#baltimore_originators  = baltimore_originators.join(respondents, on='respondentID')
bizorig_bk = bizorig

print('done')

In [ ]:
baltimore_originators = bizorig[(bizorig['state'] == 24) & (bizorig['county'] == 510)]

htmlstring = '<table>'
htmlstring += '<tr><th>Institution</th><th>Amount</th>'
for idx, amt in baltimore_originators[baltimore_originators['income_group_total'] != '   '].groupby('institution_name')['amtBizLoans'].sum().sort_values(ascending=False).iteritems():
    htmlstring += '<tr><td>{0:}</td><td>${1:,.0f}</td></tr>'.format(idx, amt)

#display(HTML(htmlstring))

#print(len(baltimore_originators))

baltimore_originators = baltimore_originators.assign(cra_level = '')

#baltimore_originators.add(pandas.Series(data=baltimore_originators['income_group'].values[0], name='cra_level'))#['income_group']
#baltimore_originators['cra_level']
#baltimore_originators.columns

baltimore_originators.loc[(baltimore_originators['income_group_total'].apply(lambda x: x in ['001', '002', '003', '004', '005'])), 'cra_level'] = 'low'
baltimore_originators.loc[(baltimore_originators['income_group_total'].apply(lambda x: x in ['006', '007', '008'])), 'cra_level'] = 'moderate'
baltimore_originators.loc[(baltimore_originators['income_group_total'].apply(lambda x: x in ['009', '010', '011', '012'])), 'cra_level'] = 'middle'
baltimore_originators.loc[(baltimore_originators['income_group_total'].apply(lambda x: x in ['013'])), 'cra_level'] = 'upper'
baltimore_originators.loc[(baltimore_originators['income_group_total'].apply(lambda x: x in ['014', '015'])), 'cra_level'] = 'unknown'

#neworder = 

#baltimore_originators = baltimore_originators.reindex(neworder)

baltimore_originators.groupby('cra_level').size().reindex(['low', 'moderate', 'middle', 'upper', 'unknown'])

#baltimore_originators[baltimore_originators['cra_level'] == '']

# Total lending in Baltimore City by institution

In [ ]:
htmlstring = '<table>'
htmlstring += '<tr><th>Institution</th><th>Amount</th>'
for idx, amt in baltimore_originators[baltimore_originators['income_group_total'] != '   '].groupby('institution_name')['amtBizLoans'].sum().sort_values(ascending=False).iteritems():
    htmlstring += '<tr><td>{0:}</td><td>${1:,.0f}</td></tr>'.format(idx, amt)

display(HTML(htmlstring))


In [ ]:

sortorderlist = baltimore_originators.groupby('institution_name')['amtBizLoans'].sum().sort_values(ascending=False).index.tolist()

sortorder = pandas.Index(sortorderlist)

lending = baltimore_originators.groupby(['institution_name', 'cra_level'])[['nBizLoans', 'amtBizLoans']].sum().unstack('cra_level').reindex(sortorder)#.sort_values(by=['amtBizLoans'], ascending=False)

lending.to_csv('lendingtable.csv', encoding='utf-8')
lending


# Get tract information

In [ ]:
print('Reading table D6...')
tracts = pandas.read_csv('cra2016_Discl_D6.dat', header=None)#, dtype={'thestring': object})
tracts.columns = ['thestring']

print('Parsing strings...')
tracts['tableID'] = tracts['thestring'].apply(lambda x: x[0:5])
tracts['respondentID'] = tracts['thestring'].apply(lambda x: x[5:15])
tracts['agency_code'] = tracts['thestring'].apply(lambda x: x[15])
tracts['activity_year'] = tracts['thestring'].apply(lambda x: x[16:20])
tracts['state'] = tracts['thestring'].apply(lambda x: x[20:22])
tracts['county'] = tracts['thestring'].apply(lambda x: x[22:25])
tracts['msa'] = tracts['thestring'].apply(lambda x: x[25:30])
tracts['census_tract'] = tracts['thestring'].apply(lambda x: x[30:37])
tracts['assessment_area_number'] = tracts['thestring'].apply(lambda x: x[37:41])
tracts['partial_county_indicator'] = tracts['thestring'].apply(lambda x: x[41])
tracts['split_county_indicator'] = tracts['thestring'].apply(lambda x: x[42])
tracts['population_classification'] = tracts['thestring'].apply(lambda x: x[43])
tracts['income_group_total'] = tracts['thestring'].apply(lambda x: x[44:47])
tracts['loan_indicator'] = tracts['thestring'].apply(lambda x: x[47])
tracts['filter'] = tracts['thestring'].apply(lambda x: x[48:145])

tracts = tracts.drop('thestring', axis=1)

print('Converting geography codes...')
tracts['respondentID'] = pandas.to_numeric(tracts['respondentID'], downcast='integer', errors='coerce')
tracts['agency_code'] = pandas.to_numeric(tracts['agency_code'], downcast='integer', errors='coerce')
tracts['activity_year'] = pandas.to_numeric(tracts['activity_year'], downcast='integer', errors='coerce')
tracts['state'] = pandas.to_numeric(tracts['state'], downcast='integer', errors='coerce')
tracts['county'] = pandas.to_numeric(tracts['county'], downcast='integer', errors='coerce')
tracts['msa'] = pandas.to_numeric(tracts['msa'], downcast='integer', errors='coerce')
tracts['census_tract'] = pandas.to_numeric(tracts['census_tract'], downcast='integer', errors='coerce')
#bizorig['assessment_area_number'] = pandas.to_numeric(bizorig['assessment_area_number'], downcast='integer', errors='coerce')

print('Looking up insitution names from respondentIDs...')
respondents = pandas.read_csv('respondentid.csv', index_col='respondentID')
tracts = tracts.join(respondents, how='left', on='respondentID')#[['respondentID', 'institution_name']].drop_duplicates()

print('Looking up data codes...')
tracts['agency'] = ''
tracts.loc[tracts['agency_code'] == 1, 'agency'] = 'OCC'
tracts.loc[tracts['agency_code'] == 2, 'agency'] = 'FRS'
tracts.loc[tracts['agency_code'] == 3, 'agency'] = 'FDIC'
tracts.loc[tracts['agency_code'] == 4, 'agency'] = 'OTS'

tracts.index.name = 'row_number'

tracts_bk = tracts


print('done')


In [ ]:
#tracts = tracts_bk

#baltimore_purchasers = bizpurch[(bizpurch['state'] == 24) & (bizpurch['county'] == 510)]
#baltimore_tracts = tracts[(tracts['state'] == 24) & tracts['county'] == 510]

#baltimore_tracts
#baltimore_originators  = baltimore_originators.join(respondents, on='respondentID')
#baltimore_purchasers  = baltimore_purchasers.join(respondents, on='respondentID')


#baltimore_tracts.groupby(['institution_name', 'census_tract']).size()

#baltimore_tracts[baltimore_tracts['loan_indicator'] == 'Y'].groupby(['institution_name', 'census_tract']).size()

#nLenders = pandas.Series(data=baltimore_tracts[baltimore_tracts['loan_indicator'] == 'Y'].groupby('census_tract', as_index=False).size().sort_values(ascending=False), name='nLenders')

#nLenders.to_csv('lenders_by_tract.csv', encoding='utf-8', header=True, index=True)

#nLenders.name = 'nLenders'

#nLenders.as_array()
#.groupby('county').size()
#baltimore_originators.to_csv('baltimore_originators.csv')
#baltimore_purchasers.to_csv('baltimore_purchasers.csv')
#baltimore_tracts.to_csv('baltimore_tracts.csv')

In [ ]:
#baltimore_tracts[baltimore_tracts['loan_indicator'] == 'Y'].groupby(['census_tract', 'institution_name']).size().sort_index()

tracts = tracts_bk

baltimore_tracts = tracts[(tracts['state'] == 24) & (tracts['county'] == 510)]

#baltimore_tracts['census_tract'].head(10)
tracts_to_neighborhoods = pandas.read_csv('neighborhoods/census_tract_to_neighborhood.csv')
tracts_to_neighborhoods = tracts_to_neighborhoods.set_index('NAME10')
#tracts_to_neighborhoods

baltimore_tracts = baltimore_tracts.join(tracts_to_neighborhoods, how='left', on='census_tract')
print('ok')


In [ ]:
loans_by_neighborhood = pandas.Series(data=baltimore_tracts[baltimore_tracts['loan_indicator'] == 'Y'].groupby(['CSA2010', 'institution_name']).size(), name='nLoans')

loans_by_neighborhood.to_csv('loans_by_neighborhood.csv', index=True, header=True)
print('ok')

In [ ]:
#baltimore_originators['income_group_total'] = pandas.to_numeric(baltimore_originators['income_group_total'], errors='coerce')
baltimore_originators[(baltimore_originators['institution_name'] == 'M&T BANK (NY)') & (baltimore_originators['cra_level'] != '')]

baltimore_originators['assessment_area_number'][(baltimore_originators['institution_name'] == 'M&T BANK (NY)') & (baltimore_originators['cra_level'] != '')]#.groupby('cra_level')[['nBizLoans1', 'nBizLoans100k', 'nBizLoans250k', 'nBizLoans1M']].sum()

#baltimore_originators['assessment_area_number']

In [ ]:
print('Reading table D1-2...')
bizpurch = pandas.read_csv('cra2016_Discl_D12.dat', header=None)#, dtype={'thestring': object})
bizpurch.columns = ['thestring']

print('Parsing strings...')
bizpurch['tableID'] = bizpurch['thestring'].apply(lambda x: x[0:5])
bizpurch['respondentID'] = bizpurch['thestring'].apply(lambda x: x[5:15])
bizpurch['agency_code'] = bizpurch['thestring'].apply(lambda x: x[15])
bizpurch['activity_year'] = bizpurch['thestring'].apply(lambda x: x[16:20])
bizpurch['loan_type'] = bizpurch['thestring'].apply(lambda x: x[20])
bizpurch['action_taken_type'] = bizpurch['thestring'].apply(lambda x: x[21])
bizpurch['state'] = bizpurch['thestring'].apply(lambda x: x[22:24])
bizpurch['county'] = bizpurch['thestring'].apply(lambda x: x[24:27])
bizpurch['msa'] = bizpurch['thestring'].apply(lambda x: x[27:32])
bizpurch['assessment_area_number'] = bizpurch['thestring'].apply(lambda x: x[33:36])
bizpurch['partial_county_indicator'] = bizpurch['thestring'].apply(lambda x: x[36])
bizpurch['split_county_indicator'] = bizpurch['thestring'].apply(lambda x: x[37])
bizpurch['population_classification'] = bizpurch['thestring'].apply(lambda x: x[38])
bizpurch['income_group_total'] = bizpurch['thestring'].apply(lambda x: x[39:42])
bizpurch['report_level'] = bizpurch['thestring'].apply(lambda x: x[42:45])
bizpurch['nBizLoans1'] = bizpurch['thestring'].apply(lambda x: x[45:55])
bizpurch['amtBizLoans1'] = bizpurch['thestring'].apply(lambda x: x[55:65])
bizpurch['nBizLoans100k'] = bizpurch['thestring'].apply(lambda x: x[65:75])
bizpurch['amtBizLoans100k'] = bizpurch['thestring'].apply(lambda x: x[75:85])
bizpurch['nBizLoans250k'] = bizpurch['thestring'].apply(lambda x: x[85:95])
bizpurch['amtBizLoans250k'] = bizpurch['thestring'].apply(lambda x: x[95:105])
bizpurch['nBizLoans1M'] = bizpurch['thestring'].apply(lambda x: x[105:115])
bizpurch['amtBizLoans1M'] = bizpurch['thestring'].apply(lambda x: x[115:125])
bizpurch['nBizLoansAff'] = bizpurch['thestring'].apply(lambda x: x[125:135])
bizpurch['amtBizLoansAff'] = bizpurch['thestring'].apply(lambda x: x[135:145])

bizpurch = bizpurch.drop('thestring', axis=1)

print('Converting geography codes...')
bizpurch['respondentID'] = pandas.to_numeric(bizpurch['respondentID'], downcast='integer', errors='coerce')
bizpurch['agency_code'] = pandas.to_numeric(bizpurch['agency_code'], downcast='integer', errors='coerce')
bizpurch['activity_year'] = pandas.to_numeric(bizpurch['activity_year'], downcast='integer', errors='coerce')
bizpurch['loan_type'] = pandas.to_numeric(bizpurch['loan_type'], downcast='integer', errors='coerce')
bizpurch['action_taken_type'] = pandas.to_numeric(bizpurch['action_taken_type'], downcast='integer', errors='coerce')
bizpurch['state'] = pandas.to_numeric(bizpurch['state'], downcast='integer', errors='coerce')
bizpurch['county'] = pandas.to_numeric(bizpurch['county'], downcast='integer', errors='coerce')
bizpurch['msa'] = pandas.to_numeric(bizpurch['msa'], downcast='integer', errors='coerce')
#bizorig['assessment_area_number'] = pandas.to_numeric(bizorig['assessment_area_number'], downcast='integer', errors='coerce')

print('Converting numbers...')
bizpurch['nBizLoans1'] = pandas.to_numeric(bizpurch['nBizLoans1'])
bizpurch['amtBizLoans1'] = pandas.to_numeric(bizpurch['amtBizLoans1']) * 1000
bizpurch['nBizLoans100k'] = pandas.to_numeric(bizpurch['nBizLoans100k'])
bizpurch['amtBizLoans100k'] = pandas.to_numeric(bizpurch['amtBizLoans100k']) * 1000
bizpurch['nBizLoans250k'] = pandas.to_numeric(bizpurch['nBizLoans250k'])
bizpurch['amtBizLoans250k'] = pandas.to_numeric(bizpurch['amtBizLoans250k']) * 1000
bizpurch['nBizLoans1M'] = pandas.to_numeric(bizpurch['nBizLoans1M'])
bizpurch['amtBizLoans1M'] = pandas.to_numeric(bizpurch['amtBizLoans1M']) * 1000
bizpurch['nBizLoansAff'] = pandas.to_numeric(bizpurch['nBizLoansAff'])
bizpurch['amtBizLoansAff'] = pandas.to_numeric(bizpurch['amtBizLoansAff']) * 1000

print('Looking up data codes...')
bizpurch['agency'] = ''
bizpurch.loc[bizpurch['agency_code'] == 1, 'agency'] = 'OCC'
bizpurch.loc[bizpurch['agency_code'] == 2, 'agency'] = 'FRS'
bizpurch.loc[bizpurch['agency_code'] == 3, 'agency'] = 'FDIC'
bizpurch.loc[bizpurch['agency_code'] == 4, 'agency'] = 'OTS'

print('Summing numbers...')
bizpurch['nBizLoans'] = bizpurch['nBizLoans1'] + bizpurch['nBizLoans100k'] + bizpurch['nBizLoans250k'] + bizpurch['nBizLoans1M'] + bizpurch['nBizLoansAff']
bizpurch['amtBizLoans'] = bizpurch['amtBizLoans1'] + bizpurch['amtBizLoans100k'] + bizpurch['amtBizLoans250k'] + bizpurch['amtBizLoans1M'] + bizpurch['amtBizLoansAff']

bizpurch.index.name = 'row_number'
print('done')

In [ ]:
baltimore_originators = bizorig[(bizorig['state'] == 24) & (bizorig['county'] == 510)]
baltimore_purchasers = bizpurch[(bizpurch['state'] == 24) & (bizpurch['county'] == 510)]
baltimore_tracts = tracts[(tracts['state'] == 24) & bizpurch['county'] == 510]

respondents = pandas.read_csv('respondentid.csv')
respondents = respondents.set_index('respondentID')

baltimore_originators  = baltimore_originators.join(respondents, on='respondentID')
baltimore_purchasers  = baltimore_purchasers.join(respondents, on='respondentID')

baltimore_originators.to_csv('baltimore_originators.csv')
baltimore_purchasers.to_csv('baltimore_purchasers.csv')
baltimore_tracts.to_csv('baltimore_tracts.csv')

In [ ]:
#from IPython.core.display import display
#from IPython.core.display import HTML
baltimore_originators.sample(2).T

x = baltimore_originators.groupby('income_group_total')['amtBizLoans'].sum().sort_index()

print('INCOME GROUP\tTOTAL AMOUNT OF LOANS IN BALTIMORE CITY')
for idx, val in x.iteritems():
    print('{0:}\t\t\t${1:,.0f}'.format(idx, val))

#baltimore_originators[baltimore_originators['institution_name'] == 'Old Line Bank (MD)'].sample(2).T
#baltimore_originators.columns